In [1]:
using Pkg
Pkg.activate("..")
#Pkg.develop(["Equilibrium", "IMAS"])
using Revise
using FUSE
using FUSE.IMAS

  Activating environment at `~/.julia/dev/FUSE/Project.toml`
┌ Info: Precompiling FUSE [e64856f0-3bb8-4376-b4b7-c03396503992]
└ @ Base loading.jl:1317


In [2]:
ϵ = 0.32
κ = 1.7
δ = 0.33
R0 = 6.2
B0 = 5.3
ip = 15.E6
beta_n = 3.0

resolution = 33

eq0=IMAS.equilibrium()
FUSE.init(eq0, 0.0; B0, R0, ϵ, δ, κ, beta_n, ip, x_point=false)

eqactor = FUSE.SolovevEquilibriumActor(eq0, 0.0, alpha=1, qstar=1.5)
@time FUSE.step(eqactor)
print(eqactor.S)
eq1 = FUSE.finalize(eqactor, resolution)

eqt = eq1.time_slice[1];

 20.931949 seconds (98.62 M allocations: 5.243 GiB, 6.57% gc time, 98.12% compilation time)
SolovevEquilibrium
  B0 = 5.3 [T]
  R0 = 6.2 [m]
  ϵ  = 0.32
  δ  = 0.33
  κ  = 1.7
  α  = 0.0027856704762293967
  q⋆ = 1.5455268252174736
  βp = 0.9981278686893219
  βt = 0.04278908932692801
  σ  = 1
  diverted  = false
  xpoint  = nothing
  symmetric = true

In [6]:
using PolygonOps
using StaticArrays
using DSP
using Random
using Interpolations
using Optim

pfactor=PFcoilsActor(eq1,12);


In [4]:
using IMAS
for expr in keys(IMAS.expressions)
    getfield(u2f(p2i(i2p(expr)[1:end-1]))(),Symbol(i2p(expr)[end]))
end